Both 2 pt and 4 pt have N systems of equations. Each system can be labelled by example element [0,0,0,k], where k=0,..,N-1

We use the alternating initial condition to predetermine the relevant systems of equations. And use dfs to generate the indicies of these systems

For alternating initial conditions only systems [0,0,0,0] and [0,0,0,N/2] have non-zero initial conditions.

In [1]:
import numpy as np
import itertools
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
N = 64
mu = 0.0
gamma = 0.0

In [3]:
##Computing Eigenvalues
E = np.zeros(N,dtype=float)
if(mu==0): #tight binding model
    print("tight binding model")
    for alpha in range(N):
        E[alpha] = 2*np.cos(2*np.pi*alpha/N)
else: #long range hopping
    for alpha in range(N):
        for j in range(1,int((N-1)/2+1)):
            E[alpha] += 2*np.cos(2*np.pi*alpha*j/N)/(j**mu)
        if (alpha%2==0): E[alpha] += 1.0/((N/2)**mu)
        else: E[alpha] += -1.0/((N/2)**mu)

tight binding model


In [5]:
def get_system(node,L,N):

    graph = {i: [] for i in range(len(L))}
    for i, node1 in enumerate(L):
        for j, node2 in enumerate(L):
            if i != j and adjacent(node1, node2, N):
                graph[i].append(j)
    
    visited = [False] * len(L)
    stack = [node]
    tree = [node]
    
    while stack:
        current = stack.pop()
        if not visited[current]:
            visited[current] = True
            for neighbor in graph[current]:
                if not visited[neighbor]:
                    stack.append(neighbor)
                    tree.append(neighbor)
    
    return tree

In [6]:
# Time points where solution is computed
logt_span = (-2, +2)
log_times = np.linspace(-2, +2, 100)

In [ ]:
# progress bar for integration
def progress_bar(t,y):
    pbar.update(1)

## 2 Point correlators

In [7]:
def adjacent(arr1, arr2, N):
    i1, i2 = arr1
    j1, j2 = arr2
    if (i1+i2-j1-j2)%N==0:
        return True
    return False

In [ ]:
def ode_coeff_2pt(arr1,arr2):
    p1,p2=arr1
    q1,q2=arr2
    if(arr1==arr2): return 1.0j*(E[p1]-E[p2]) - gamma*(1-1/N)
    else:
        if (p1+p2-q1-q2)%N==0 : return gamma/N + 0.0j

In [ ]:
def equation_2pt(i,S):
    n = len(S)
    return sum(ode_coeff_2pt(system[i],system[j])*S[j] for j in range(n))

In [ ]:
def odes_2pt(t, S1):
    n = len(S1)
    dxdt = np.zeros(n,dtype=complex)
    for i in range(n):
        dxdt[i] = (10**t)*equation_2pt(i, S1)
    return dxdt

In [ ]:
D_mm = np.zeros(len(log_times),dtype=complex)

L = list(itertools.product(range(N), repeat=2))
for system in [0,int(N/2)]: #is determined by alternating initial state
    system = list(set(get_system(system,L,N)))
    system = [L[i] for i in system] #list of mode touples that are in this system of equations
    n = len(system)

    print(f"system {system[0]} defined. size: {n}")

    #initial condition
    d0 = [0.5 + 0.0j for i in range(n)]

    solution = solve_ivp(odes_2pt, logt_span, d0, t_eval=log_times,method='RK45')
    
    #fourier scaling
    four_scaling = [sum(np.exp(2.0j*np.pi*m*sum(system[k])/N) for m in range(int(N/2))) for k in range(n)]
    #scaled_solution = [four_scaling(k)*solution.y[k,:] for k in range(n)]
    #D_mm += [sum(scaled_solution[:,t]) for t in range(len(log_times))]
    D_mm += [sum(four_scaling[k]*solution.y[k][t] for k in range(n))/(N) for t in range(len(log_times))]

## 4 point

In [ ]:
def adjacent(arr1, arr2, N):
    i1, i2, i3, i4 = arr1
    j1, j2, j3, j4 = arr2
    if i1 == j1:
        if i3 == j3 and (i2 + i4 - j2 - j4) % N == 0:
            return True
        if i4 == j4 and (i2 + i3 - j2 - j3) % N == 0:
            return True
    if i2 == j2:
        if i3 == j3 and (i1 + i4 - j1 - j4) % N == 0:
            return True
        if i4 == j4 and (i1 + i3 - j1 - j3) % N == 0:
            return True
    return False

In [ ]:
def ode_coeff_4pt(arr1,arr2):
    p1,p2,p3,p4=arr1
    q1,q2,q3,q4=arr2
    coeff = 0.0 + 0.0j
    if(arr1==arr2): return 1.0j*(E[p1]+E[p2]-E[p3]-E[p4]) - gamma*(2-1/(4*N))
    else:
        if p1 == q1:
            if p3 == q3 and (p2 + p4 - p2 - p4) % N == 0:
                coeff += gamma/N
            if p4 == q4 and (p2 + p3 - q2 - q3) % N == 0:
                coeff += gamma/N
        if p2 == q2:
            if p3 == q3 and (p1 + p4 - q1 - q4) % N == 0:
                coeff += gamma/N
            if p4 == q4 and (p1 + p3 - q1 - q3) % N == 0:
                coeff += gamma/N
    return coeff

In [ ]:
def equation_4pt(i,S):
    n = len(S)
    return sum(ode_coeff_4pt(system[i],system[j])*S[j] for j in range(n))

In [ ]:
def odes_4pt(t, S1):
    n = len(S1)
    dxdt = np.zeros(n,dtype=complex)
    for i in range(n):
        dxdt[i] = (10**t)*equation_4pt(i, S1)
    return dxdt

In [ ]:
F_mnmn = np.zeros(len(log_times),dtype=complex)

L = list(itertools.product(range(N), repeat=4))
for system in [0,int(N/2)]: #is determined by alternating initial state

    system = list(set(get_system(system,L,N)))
    system = [L[i] for i in system] #list of mode touples that are in this system of equations
    n = len(system)
    print(f"system {system[0]} defined. size: {n}")

    #initial condition
    f0 = np.zeros(n,dtype=complex)
    for i in range(n):
        k1,k2,k3,k4 = system[i]
        if (k1+k3)%(N/2) == 0:
            if(k2+k4)%(N/2) == 0: f0[i] -= 0.25
        if (k1+k4)%(N) == 0:
            if(k2+k3)%(N) == 0: f0[i] += 0.25
    print("Initial state defined")
    

    solution = solve_ivp(odes_4pt, logt_span, f0, t_eval=log_times,method='RK45')
    
    #fourier scaling:
    four_scaling = [sum(np.exp(2.0j*np.pi*(m*(system[k][0]+system[k][2])+n*(system[k][1]+system[k][3]))) for m in range(int(N/2)) for n in range(int(N/2))) for k in range(n)]
    #scaled_solution = [four_scaling(k)*solution.y[k,:]/(N**4) for k in range(n)]
    #F_mnmn += [sum(scaled_solution[:,t]) for t in range(len(log_times))]
    F_mnmn += [sum(four_scaling(k)*solution.y[k][i] for k in range(n))/(N**4) for i in range(len(log_times))]

In [ ]:
w_sq = [(-F_mnmn[t]+D_mm[t]-(D_mm[t]**2)) for t in range(len(log_times))]

In [ ]:
import pandas as pd